# CSC421 Fall 2021 Assignment 2 
### Author: George Tzanetakis 

This notebook is based on the supporting material for topics covered in **Chapter 7 - Logical Agents** and **Chapter 8 - First-Order Logic** from the book *Artificial Intelligence: A Modern Approach.* You can consult and modify the textbook code provided in logic.py and logic.ipynb for completing the assignment questions. The first 5 questions do NOT rely on the provided aima code so you can complete it just using basic Python. The next 5 questions require the use of the aima-code repository. 

You are welcome and actually it can be educational to look at the code at the aima-code repository as well as other code resources you can find on the web. However, make sure you understand any code that you incoporate. 

The assignment structure is as follows - each item is worth 1 point: 

1. Propositional Logic (Basic) - simple infix evaluator for 0, 1 and logical operators 
2. Propositional Logic (Basic) - adding variables and bindings to evaluator 
3. Propositional Logic (Expected) - recursive infix evaluator for propositional logic 
4. Propositional Logic (Expected) - usage of evaluator to evaluate some example logic expressions
5. Propositional Logic (Advanced) - model checking for the prefix evaluator 
6. Propositional Logic(Basic) - simple KB using aima code with model checking and theorem proving 
7. First-Order Logic (Basic) - kinship domain using FolKB for the Simpsons 
8. First-Order Logic (Expected) - kinship domain using Prolog for the Simpsons
9. First-Order Logic (Expected) - LegoWorld 
10.First-Order Logic (Advanced) - NLP for LegoWorld or webscraping a KB for music  

The grading will be done in increments of 1. 2 points for correct answer, 1 points for partial or incorrect but reasonable answer and 0.0 for no answer or completely wrong answer. 

```
Birds can fly, unless they are penguins and ostriches, or if they happen 
to be dead, or have broken wings, or are confined to cages, or have their 
feet stuck in cement, or have undergone experiences so dreadful as to render 
them psychologically incapable of flight 

Marvin Minsky 
```

# Introduction - Parsing and evaluating prefix logic expressions  

In this assignment your task is to incrementally create a parser and evaluator for prefix logic expressions as well as implement simple model checking. 


# Question 1 (Basic)  - 2 points

Your first task will be to write a simple evaluator of prefix logic expressions with constants. In prefix notation the operator precedes the operands and no operands are required. For example 5+3 in prefix notation is written + 5 3 or 5 * 2 + 3 would be written + * 5 2 3 or + * 5 2 * 3 4 is equivalent to (5 * 2) + (3 * 4). 

As a first step we will consider very simple expressions with one operator and two constant operands. We will use 0 for false and 1 for true. The following logical connectives should be implemented (see Figure 7.8 in your book) (notice that for now there is no negation symbol ~): 

1. &    (and), 
3. |    (or), 
4. =>   (implication) 
5. <=>  (biconditional) 

Example expressions: 
```
& 1 0  
=> 0 1 
<=> 1 1 
```

Your function should take as input a string with the prefix expression and return the result of evaluating the expression (basically a 1 for true and 0 for false). You can split a string to a list using .split[' ']. For this part of the assignment you only evaluate expressions with two constant operands i.e no nested/recursive expressions. 

In [59]:
a = '& 1 0'
print(a.split(' '))

['&', '1', '0']


In [60]:
# YOUR CODE GOES HERE 
def logic_expression(expression):
    expressions = expression.split(' ')
    operation = expressions[0]
    x = int(expressions[1])
    y = int(expressions[2])

    if operation == '&':
        return x and y
    elif operation == '|':
        return x or y
    elif operation == '=>':
        return (1-x) or y
    elif operation == '<=>':
        return (x and y) or ((1-x) and (1-y))


# test
test = ['& 1 1', '& 1 0', '& 0 1', '& 0 0',
        '| 1 1', '| 1 0', '| 0 1', '| 0 0',
        '<=> 1 1', '<=> 1 0', '<=> 0 1', '<=> 0 0',
        '=> 1 1', '=> 1 0', '=> 0 1', '=> 0 0']
print('------------------------------q1---------------------------------')
for i in test:
    print(i, '-', logic_expression(i))

------------------------------q1---------------------------------
& 1 1 - 1
& 1 0 - 0
& 0 1 - 0
& 0 0 - 0
| 1 1 - 1
| 1 0 - 1
| 0 1 - 1
| 0 0 - 0
<=> 1 1 - 1
<=> 1 0 - 0
<=> 0 1 - 0
<=> 0 0 - 1
=> 1 1 - 1
=> 1 0 - 0
=> 0 1 - 1
=> 0 0 - 1


# Question 2 (Basic) - 2 points

Your next task is to implement variables and bindings for your propositional logic evaluator. In this version in addition to constants (0 and 1) you also can have variables which are strings with associated values provided in a dictionary. You still only consider two operands and one operator (no nesting). For example in the code below 
the three expressions are equivalent. Your function should take as arguments the expression to be evaluated as a string and the dictionary with the variable bindings. In addition you need to add the ~ (not) operator. To do so for each variable in the dictionary add a not version. For example if 'a' in the dictionary has a value of 1 the '~a' in the dictionary should have a value of 0. Notice that the not symbol is part of the string and NOT separated by a space. 



In [61]:
d = {'foo': 0, 'b': 1}
print(d)
expr1 = '& 0 1'
expr2 = '& foo 1'
expr3 = '& foo ~b'

{'foo': 0, 'b': 1}


In [62]:
# YOUR CODE GOES HERE 
def generate_full_dictionary(dictionary):
    full_dic = {}
    for key in dictionary.keys():
        if key[0] == '~':
            full_dic[key] = dictionary[key]
            full_dic[key[1:]] = 1-dictionary[key]
        elif key == '1' or key == '0':
            continue
        else:
            full_dic[key] = dictionary[key]
            full_dic['~'+key] = 1-dictionary[key]

    full_dic['1'] = 1
    full_dic['0'] = 0

    return full_dic


def logic_expression_with_dict(expression, dictionary):
    full_dict = generate_full_dictionary(dictionary)
    expressions = expression.split(' ')
    operation = expressions[0]
    x = full_dict[expressions[1]]
    y = full_dict[expressions[2]]

    if operation == '&':
        return x and y
    elif operation == '|':
        return x or y
    elif operation == '=>':
        return (1-x) or y
    elif operation == '<=>':
        return (x and y) or ((1-x) and (1-y))


# test
print('------------------------------q2---------------------------------')
d = {'foo': 0, 'b': 1, '~a': 0, '1': 1, '0': 0}
d2 = generate_full_dictionary(d)
# for i in d2:
#     print(i,d2[i])

d = {'foo': 0, 'b': 1}
expr1 = '& 0 1'
expr2 = '& foo 1'
expr3 = '& foo ~b'

expr4 = '| 0 1'
expr5 = '| ~foo b'
expr6 = '| ~foo ~b'
expr7 = '| foo b'
expr8 = '| foo ~b'

expr9 = '=> 0 1'
expr10 = '=> ~foo b'
expr11 = '=> ~foo ~b'
expr12 = '=> foo b'
expr13 = '=> foo ~b'

expr14 = '<=> 1 0'
expr15 = '<=> ~foo b'
expr16 = '<=> ~foo ~b'
expr17 = '<=> foo b'
expr18 = '<=> foo ~b'

print(expr1, logic_expression_with_dict(expr1, d))
print(expr2, logic_expression_with_dict(expr2, d))
print(expr3, logic_expression_with_dict(expr3, d))
print('___________________')
print(expr4, logic_expression_with_dict(expr4, d))
print(expr5, logic_expression_with_dict(expr5, d))
print(expr6, logic_expression_with_dict(expr6, d))
print(expr7, logic_expression_with_dict(expr7, d))
print(expr8, logic_expression_with_dict(expr8, d))
print('___________________')
print(expr9, logic_expression_with_dict(expr9, d))
print(expr10, logic_expression_with_dict(expr10, d))
print(expr11, logic_expression_with_dict(expr11, d))
print(expr12, logic_expression_with_dict(expr12, d))
print(expr13, logic_expression_with_dict(expr13, d))
print('___________________')
print(expr14, logic_expression_with_dict(expr14, d))
print(expr15, logic_expression_with_dict(expr15, d))
print(expr16, logic_expression_with_dict(expr16, d))
print(expr17, logic_expression_with_dict(expr17, d))
print(expr18, logic_expression_with_dict(expr18, d))

------------------------------q2---------------------------------
& 0 1 0
& foo 1 0
& foo ~b 0
___________________
| 0 1 1
| ~foo b 1
| ~foo ~b 1
| foo b 1
| foo ~b 0
___________________
=> 0 1 1
=> ~foo b 1
=> ~foo ~b 0
=> foo b 1
=> foo ~b 1
___________________
<=> 1 0 0
<=> ~foo b 1
<=> ~foo ~b 0
<=> foo b 0
<=> foo ~b 1


# Question 3 (Expected) 2 points 


The following code is a recursive evaluator for prefix arithmetic expressions. It assumes that there are always two operands either an integer or a prefix expression starting with an operator (addition or multiplication). It is a good idea to go through this function carefully by hand to understand how the recursion works. 

Informed by your understanding of the arithmetic recursive_eval function your task is to write function to implement a recursive prefix logic evaluator. Your evaluator should also support variables bindings using a dictionary as in the previous question. 

Example expressions: 
```
& 1 & 1 a   
=> 0 & b ~alice  
<=> foo 1 
```

In [63]:
def recursive_eval(l):
    head, tail = l[0], l[1:]
    if head in ['+', '*']: 
        val1, tail = recursive_eval(tail)
        val2, tail = recursive_eval(tail)
        if head == '+': 
            return (int(val1)+int(val2), tail)
        elif head == '*':  
            return (int(val1)*int(val2), tail)
    # operator is a number 
    else:  
        return (int(head),tail)

def prefix_eval(input_str): 
    input_list = input_str.split(' ')
    res, tail = recursive_eval(input_list)
    return res

print(prefix_eval('1'))
print(prefix_eval('+ 1 2'))
print(prefix_eval('+ 1 * 2 3'))
print(prefix_eval('+ * 5 2 * 3 + 1 5'))

1
3
7
28


In [64]:
# YOUR CODE GOES HERE 
def recursive_eval_l(expression, dictionary):
    head, tail = expression[0], expression[1:]

    # head is a operator
    if head in ['&', '|', '=>', '<=>']:
        val1, tail = recursive_eval_l(tail, dictionary)
        val2, tail = recursive_eval_l(tail, dictionary)
        if head == '&':
            return (val1 and val2, tail)
        elif head == '|':
            return (val1 or val2, tail)
        elif head == '=>':
            return ((1-val1) or val2, tail)
        elif head == '<=>':
            return ((val1 and val2) or ((1-val1) and (1-val2)), tail)
    # head is a number
    else:
        return (dictionary[head], tail)


def prefix_eval_l(expression, dictionary):
    input_list = expression.split(' ')
    full_dictionary = generate_full_dictionary(dictionary)
    res, tail = recursive_eval_l(input_list, full_dictionary)
    return res


# test
print('------------------------------q3---------------------------------')
d = {'foo': 0, 'b': 1, 'alice': 0, 'a': 0}
expr19 = '=> 0 & b ~alice'
expr20 = '& 1 & 1 a'
expr21 = '=> 0 & b ~alice'
expr22 = "<=> foo 1"

print(prefix_eval_l(expr19, d))
print(prefix_eval_l(expr20, d))
print(prefix_eval_l(expr21, d))
print(prefix_eval_l(expr22, d))

------------------------------q3---------------------------------
1
0
1
0


# QUESTION 4 (EXPECTED) - 2 points


Using the recursive prefix evaluator you defined in the previous question 
answer the following question (you will need to convert the exressions below 
to prefix). You can use multiple string assignments to assemble more complicated 
sentences into one big string: 


Let A be the formula: 

\begin{equation} 
  (( p_{1} \rightarrow (p2 \land p_{3})) \land ((\neg p_{1})
  \rightarrow (p_{3} \land p_{4})))
\end{equation} 

Let B be the formula: 

\begin{equation} 
  (( p_{3} \rightarrow (\neg p_{6})) \land ((\neg
  p_{3}) \rightarrow (p_{4} \rightarrow p_{1})))  
\end{equation} 

Let C be the formula: 

\begin{equation} 
  ((\neg(p2 \land p_{5})) \land (p2 \rightarrow p_{5})) 
\end{equation} 

Let D be the formula: 

\begin{equation} 
  (\neg (p_{3} \rightarrow p_{6}))
\end{equation} 

Evaluate the formulate E: 
\begin{equation} 
  (( A \land (B \land C)) \rightarrow D)
\end{equation} 

under the true assignment $I_{1}$, where $I_{1}(p_{1}) = I_{1}(p_{3}) = I_{1}(p_{5}) = false$ 
and $I_{1}(p2) = I_{1}(p_{4}) = I_{1}(p_{6}) = true$ as well as under the truth assignment 
$I_{2}$, where $I_{2}(p_{1}) = I_{2}(p_{3}) = I_{2}(p_{5}) = true$ and
$I_{2}(p_{2})=I_{2}(p_{4})=I_{2}(p_{6}) = false$. 


In [65]:
## YOUR CODDE GOES HERE 
print('------------------------------q4---------------------------------')

print('Assignment  I1')
d = {'p1': 0, 'p2': 1, 'p3': 0, 'p4': 1, 'p5': 0, 'p6': 1}
A = prefix_eval_l('& => p1 & p2 p3 => ~p1 & p3 p4', d)
B = prefix_eval_l('& => p3 ~p6 => ~p3 => p4 p1', d)
C = prefix_eval_l('& | ~p2 ~p5 => p2 p5', d)
# ~(p3=>p6)
# = ~(~p3 | p6)
# = ~~p3 & ~p6
# = & p3 ~p6
D = prefix_eval_l('& p3 ~p6', d)

d['A'] = A
d['B'] = B
d['C'] = C
d['D'] = D

E = prefix_eval_l('=> & A & B C D', d)

print("A:", A)
print("B:", B)
print("C:", C)
print("D:", D)
print("E:", E)
print('Assignment  I1:', E)

print('----------')

print('Assignment I2')
d = {'p1': 1, 'p2': 0, 'p3': 1, 'p4': 0, 'p5': 1, 'p6': 0}
A = prefix_eval_l('& => p1 & p2 p3 => ~p1 & p3 p4', d)
B = prefix_eval_l('& => p3 ~p6 => ~p3 => p4 p1', d)
C = prefix_eval_l('& | ~p2 ~p5 => p2 p5', d)
D = prefix_eval_l('& p3 ~p6', d)

d['A'] = A
d['B'] = B
d['C'] = C
d['D'] = D

E = prefix_eval_l('=> & A & B C D', d)

print("A:", A)
print("B:", B)
print("C:", C)
print("D:", D)
print("E:", E)
print('Assignment  I2:', E)

------------------------------q4---------------------------------
Assignment  I1
A: 0
B: 0
C: 0
D: 0
E: 1
Assignment  I1: 1
----------
Assignment I2
A: 0
B: 1
C: 1
D: 1
E: 1
Assignment  I2: 1


# QUESTION 5 (ADVANCED) - 2 points 

Implement inference using model-checking using your prefix recursive evaluator to decide whether a knowledge base KB entais some sentence a. To do so express the knowledge base in the prefix notation, enumerate all models for the variables in the dictionary, and check that the sentence a is true in every model in which the KB is true. 

You can check the implementation to tt_entails in logic.ipynb in the aima_python repository to inform how you implement your solution. Your solution should NOT rely directly on any code in logic.py or logic.ipynb. 

Check your model checking using the examples that are used in logic.ipynb to check entailment (there are a few with P and Q as variables as well as the one with A, B, C, D, E, F, G. You will need to convert these examples to prefix notation. 


In [66]:
# YOUR CODE GOES HERE 
from itertools import product

def return_answer(not_operator: bool, expression: bool, tail):
    if not_operator:  # reverse
        return (not expression, tail)
    else:
        return (expression, tail)


def recursive_eval_l_2(expression, dictionary):
    head, tail = expression[0], expression[1:]

    # head is a operator
    if head in ['&', '|', '=>', '<=>']:
        val1, tail = recursive_eval_l_2(tail, dictionary)
        val2, tail = recursive_eval_l_2(tail, dictionary)
        if head == '&':
            return (val1 and val2, tail)
        elif head == '|':
            return (val1 or val2, tail)
        elif head == '=>':
            return ((1-val1) or val2, tail)
        elif head == '<=>':
            return ((val1 and val2) or ((1-val1) and (1-val2)), tail)
    # head is a number
    else:
        return (dictionary[str(head)], tail)


def prefix_eval_logic(input_str: str, sentense: str):
    print('-----------------------------')
    input_list = input_str.replace('(', '').replace(')', '').split(' ')
    sentense_list = sentense.replace('(', '').replace(')', '').split(' ')
    dictionary_models = generate_variable_dictionary(input_list)

    for dictionary_model in dictionary_models:
        new_dictionary = generate_full_dictionary(dictionary_model)
        model_assignment = [char for char in input_list]
        model_assignment_sentense = [char for char in sentense_list]

        for i, key in enumerate(model_assignment):
            if key in new_dictionary:
                model_assignment[i] = new_dictionary[key]
            elif key == '1' or key == '0':
                model_assignment[i] = int(key)

        res_kb, tail = recursive_eval_l_2(model_assignment, new_dictionary)

        for i, key in enumerate(model_assignment_sentense):
            if key in new_dictionary:
                model_assignment_sentense[i] = new_dictionary[key]
            elif key == '1' or key == '0':
                model_assignment_sentense[i] = int(key)
        res_sentese, tail_sentense = recursive_eval_l_2(
            model_assignment_sentense, new_dictionary)

        if res_kb != res_sentese:
            return False

    # if all models result are the same, then we can say KB entails the sentense :)
    return True
# ---------------------------------


def generate_variable_dictionary(input_list):

    # 1.count number of variables
    seen_variables = set([])
    for key in input_list:
        if key not in ['&', '|', '=>', '<=>']:
            seen_variables.add(key)
    print("all variables:", seen_variables)

    output = []

    zero_one_combinations = list(product(range(2), repeat=len(seen_variables)))
    for i, zero_one_tuple in enumerate(zero_one_combinations):
        # print("zero_one_tuple:",zero_one_tuple)
        dictionary = {}
        for var, zero_one in zip(list(seen_variables), list(zero_one_tuple)):
            dictionary[var] = zero_one
        output.append(dictionary)

    print("all models:", output)
    return output


knowledge_base = '& (=> p1 p2) (=> p3 p1)'
sentense = '=> p1 p2'

print('test1: KB entails sentense?:', 'KB=', knowledge_base, 'sentense=', sentense, 'entails= ',prefix_eval_logic(knowledge_base, sentense))

knowledge_base2 = '& (=> p1 p2) (=> p2 p1)'
sentense2 = '<=> p1 p2'
print('test2: KB entails sentense?:', 'KB2=', knowledge_base2, 'sentense2=', sentense2, 'entails= ',prefix_eval_logic(knowledge_base2, sentense2))

-----------------------------
all variables: {'p1', 'p2', 'p3'}
all models: [{'p1': 0, 'p2': 0, 'p3': 0}, {'p1': 0, 'p2': 0, 'p3': 1}, {'p1': 0, 'p2': 1, 'p3': 0}, {'p1': 0, 'p2': 1, 'p3': 1}, {'p1': 1, 'p2': 0, 'p3': 0}, {'p1': 1, 'p2': 0, 'p3': 1}, {'p1': 1, 'p2': 1, 'p3': 0}, {'p1': 1, 'p2': 1, 'p3': 1}]
test1: KB entails sentense?: KB= & (=> p1 p2) (=> p3 p1) sentense= => p1 p2 entails=  False
-----------------------------
all variables: {'p1', 'p2'}
all models: [{'p1': 0, 'p2': 0}, {'p1': 0, 'p2': 1}, {'p1': 1, 'p2': 0}, {'p1': 1, 'p2': 1}]
test2: KB entails sentense?: KB2= & (=> p1 p2) (=> p2 p1) sentense2= <=> p1 p2 entails=  True


# Extra ideas (no credit) 

* Implement conversion of the prefix expressions to prefix conjuctive normal form (CNF) based on the recursive evaluator you have implemented. 
* Based on the recursive evaluator you have implemented do a conversion of expressions in prefix notation to the infix notation of expressions supported by logic.ipynb. Provide 4 test cases that demonstrate the the conversion works by confirming that the result of your evaluator and the logic.ipynb evaluator are the same. 



# Question 6 (Basic) -2 points

Consider the following propositional logic knowledge base.

It is not sunny this afternoon and it is colder than yesterday.
We will go swimming only if it is sunny.
If we do not go swimming then we will take a canoe trip.
If we take a canoe trip, then we will be home by sunset.
Denote:

* p = It is sunny this afternoon
* q = it is colder than yesterday
* r = We will go swimming
* s= we will take a canoe trip
* t= We will be home by sunset

Express this knowledge base using propositional logic using the expression syntax used in logic.ipynb. You can incoprorate any code you need from logic.ipynb and logic.py. Using both model checking and theorem proving inference (you can use the implementations providedin logic.py) show that this knowledge base entails the sentence if it is not sunny this afternoon then we will be home by sunset. 

In [69]:
# YOUR CODE GOES HERE 
%cd ~
%cd "aima-python"
from utils import *
from logic import *

my_kb=PropKB()
(p,q,r,s,t)=expr('p,q,r,s,t')
my_kb.tell(~p)
my_kb.tell(q)
my_kb.tell(r |'==>'|p)
my_kb.tell(~r |'==>'|s)
my_kb.tell(s |'==>'|t)

my_kb.clauses
pl_resolution(my_kb,~p |'==>'|t)
# print(my_kb.clauses)


C:\Users\Administrator
C:\Users\Administrator\aima-python


True

# Question 7 (Basic)  - 2 points

Encode the kindship domain described in section 8.3.2 of the textbook using FOL and FolKB implementation in logic.py and encode as facts the relationships between the members of the Simpsons family from the popular TV show:  

https://en.wikipedia.org/wiki/Simpson_family


Show how the following queries can be answered using this KB: 

* Who are the children of Homer ? 
* Who are the parents of Bart ? 
* Are Lisa and Homer siblings ? 
* Are Lisa and Bart siblings ? 


In [70]:
# YOUR CODE GOES HERE 
clauses = []

clauses.append(expr("Parents(Homer,Lisa)"))
clauses.append(expr("Parents(Homer,Bart)"))
clauses.append(expr("Parents(Homer,Maggie)"))
clauses.append(expr("Parents(Homer,Hugo)"))
clauses.append(expr("Parents(Marge,Lisa)"))
clauses.append(expr("Parents(Maege,Bart)"))
clauses.append(expr("Parents(Marge,Maggie)"))
clauses.append(expr("Parents(Marge,Hugo)"))

clauses.append(expr("Parents(p,s) & Parents(p,d)==>Sibling(s,d)"))
clauses.append(expr("Children(c,p)==>Parents(p,c)"))



S_kb=FolKB(clauses)
print("Who are the children of Homer ?")
answer=fol_fc_ask(S_kb,expr('Parents(Homer,x)'))
print(list(answer))
print("Who are the parents of Bart ?")
answer=fol_fc_ask(S_kb,expr('Parents(x,Bart)'))
print(list(answer))
print("Are Lisa and Homer siblings ?")
answer=S_kb.ask(expr("Sibling(Lisa, Homer)"))
if answer!=False:
    answer=True
print(answer)
print("Are Lisa and Bart siblings ?")
answer=S_kb.ask(expr("Sibling(Bart, Lisa)"))
if answer!=False:
    answer=True
print(answer)

Who are the children of Homer ?
[{x: Lisa}, {x: Bart}, {x: Maggie}, {x: Hugo}]
Who are the parents of Bart ?
[{x: Homer}, {x: Maege}]
Are Lisa and Homer siblings ?
False
Are Lisa and Bart siblings ?
True


# QUESTION 8 (EXPECTED) 2 points

In this question we explore Prolog which is a programming language based on logic. We won't go into details but just wanted to give you a flavor of the syntax and how it connects to what we have learned. For this question you 
will NOT be using the notebook so your answer should just be the source code. We will use http://tau-prolog.org/ which is a Prolog implementation that can run in a browser. When you access the webpage there is a text window labeled try it for entering your knowledge base and under it t
here is a text entry field for entering your query. 

For example type in the Try it window and press enter: 

```Prolog
likes(sam, salad).
likes(dean, pie).
likes(sam, apples).
likes(dean, whiskey).
```

Then enter the query: 
```Prolog 
likes(sam,X).
```
When you press Enter once you will get X=apples. and X=salad. Note the periods at the end of each statement. 

Encode the kinship domain from the previous question in Prolog and answer the queries from the previous question. Notice that in Prolog the constants start with lower case letters and the variables start with upper case letters.

Provide your code for the KB and queries using markup. See the syntax for Prolog of this cell by double clicking for editing. 



# YOUR CODE GOES HERE (USING MARKDOWN)
mother(marge, hugo).
mother(marge, bart). 
mother(marge, lisa). 
mother(marge, maggie). 
father(homer, hugo).
father(homer, bart). 
father(homer, lisa). 
father(homer, maggie).

parent(P, C) :- father(P, C). 
parent(P, C) :- mother(P, C). 
sibiling(X, Y) :- parent(P, X), parent(P, Y).
child(C, P) :- parent(P, C). 



child(X,marge).
X = maggie.
X = lisa ;
X = bart ;
X = hugo ;


parent(X,bart).
X = marge.
X = homer ;


sibiling(homer, lisa).
false


sibiling(lisa, bart).
true

# QUESTION 9 (EXPECTED) 2 points


## Legoworld 


In this question we explore the use of FOL to encode knowledge about the objects 
and arrangement of a simple world created by different lego pieces. Our world 
will consist of making simple structure by placing lego pieces on top of each other. 
Each lego piece will be identified by a unique identifier (the letters in the figure below). 

Let's look at a specific example where each piece is labeled by a letter: 

<img src="lego_letters.png" width="60%"/>

This corresponds to the following picture: 

<img src="lego2.png" width="60%"/>



We can use the following predicates to model the world: 
* OnPlate(p): p is attached to the bottom plate 
* On(p1,p2): piece p1 is placed on top of piece p2 
* AtLeft(p1,p2): piece p1 and piece p2 are placed on the plate, and piece p1 is direct at the left of p2 
* Color(p,c): The color of piece p is c (Red, Grey, Brown, White, Yellow, Blue) 
* Type(p,t): The type of piece p is t (Brick, Plate, Tile) 

<img src="lego3.png" width="20%"/>


Each pieces will be identified by the letters appearing in the picture. The thicker brick with studs will have type  Brick, the thinner brick with studs is of type Plate, and the one that is flat on the top is of type Tile. 


Use the FO KB implementation in logic.py to: 

1. Write a database of facts which models the world in the picture. For example you can use clauses.append(expr('TypeOf(A,Brick)')) to state that lego piece A is a Brick. 

2. Based exclusively on using these predicates (OnPlate, On, AtLeft), define the following predicates:
    * Base(b1, b2): b2 is the base of the tower containing b1.
    * Base_at_right(b1, b2): b1 and b2 are on the plate, and b2 is at the right (but perhaps not directly) of b1.
    * Object_at_right(b1, b2): b1 is in a pile which is at the right (but perhaps not directly) of b2.
    
    
The above predicates must work for any world defined using the facts specified by on_plate, on, 
at_left not just the specific example encoded above. In other words these predicates should be defined 
in terms of the existing predicates and variables. As an example here is the definition of Base. 

    * clauses.append(expr('OnPlate(x) ==> Base(x,x)'))
    * clauses.append(expr('On(x,z) & Base (z,y) ==> Base(x,y)'))
    
This is a recursive definition it is a good idea to see how it works by doing substitutions by hand. 


3. Using the KB you created answer the following queries: 
    * Is piece B on top of piece C? 
    * What is the type and color of the piece on top of C? 
    * What is the type of the base of H? 
    * What are the bricks that are right of C? 
    * What are all the bricks that are on top of i ? 






In [71]:
print('------------------------------q9---------------------------------')

clauses=[]
BLOCKS =['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

clauses.append(expr('TypeOf(A, Brick)'))
clauses.append(expr('TypeOf(B, Tile)'))
clauses.append(expr('TypeOf(C, Plate)'))
clauses.append(expr('TypeOf(D, Brick)'))
clauses.append(expr('TypeOf(E, Tile)'))
clauses.append(expr('TypeOf(F, Brick)'))
clauses.append(expr('TypeOf(G, Tile)'))
clauses.append(expr('TypeOf(H, Brick)'))
clauses.append(expr('TypeOf(I, Brick)'))
clauses.append(expr('TypeOf(J, Plate)'))

# COLOR
clauses.append(expr('Color(A, Red)'))
clauses.append(expr('Color(B, White)'))
clauses.append(expr('Color(C, Brown)'))
clauses.append(expr('Color(D, Grey)'))
clauses.append(expr('Color(E, Brown)'))
clauses.append(expr('Color(F, Red)'))
clauses.append(expr('Color(G, Brown)'))
clauses.append(expr('Color(H, Red)'))
clauses.append(expr('Color(I, Blue)'))
clauses.append(expr('Color(J, Yellow)'))

# ONPLATE
clauses.append(expr('OnPlate(A)'))
clauses.append(expr('OnPlate(D)'))
clauses.append(expr('OnPlate(F)'))
clauses.append(expr('OnPlate(J)'))

# ON WITH CONNRCTION
clauses.append(expr('On(B, C)'))
clauses.append(expr('On(C, D)'))
clauses.append(expr('On(E, F)'))
clauses.append(expr('On(G, H)'))
clauses.append(expr('On(H, I)'))
clauses.append(expr('On(I, J)'))

# ON the left
clauses.append(expr('AtLeft(A, D)'))
clauses.append(expr('AtLeft(D, F)'))
clauses.append(expr('AtLeft(F, J)'))


ab3 = expr('(On(v,x) & On(x,y) & On(y,z) ==> Above(v,z))')
ab2 = expr('(On(x,y) & On(y,z) ==> Above(x,z))')
ab1 = expr('(On(x,y) ==> Above(x,y))')

# Base
clauses.append(expr('OnPlate(x) ==> Base(x,x)'))
clauses.append(expr('On(x,z) & Base (z,y) ==> Base(x,y)'))

# Base_at_right
clauses.append(expr('AtLeft(a, b) ==> Base_at_right(a, b)'))
clauses.append(expr('AtLeft(b, c) & Base_at_right(a, b) ==> Base_at_right(a, c)'))

# Object_at_right
clauses.append(expr('Base(x,a) & Base(y,b) & Base_at_right(a, b) ==> Object_at_right(x, y)'))

clauses.append(ab3)
clauses.append(ab2)
clauses.append(ab1)

lego_kb=FolKB(clauses)


#########################################################
print("Is piece B on top of piece C?")
ans=True if lego_kb.ask(expr('On(B,C)'))=={} else False
print(f"computer:{ans}")
print('-'*20)
#########################################################
print('What is the type and color of the piece on top of C?')
target='C'  # modify here
search=lego_kb.ask(expr(f'On(x,{target})'))
upperblock=search if search!={} else False
if upperblock:
    upper_block=search[x]
    upper_block_color=lego_kb.ask(expr(f'Color({upper_block},y)'))[y]
    upper_block_type=lego_kb.ask(expr(f'TypeOf({upper_block},z)'))[z]
    print(f'It\'s a {upper_block_type} and it\'s {upper_block_color}')
else:
    print('There is no top on the block')
print('-'*20)
#########################################################
print('What is the type of the base of H? ')
target='H'  # modify here
search=lego_kb.ask(expr(f'Base({target},x)'))
# print(search)
baseblock = search[x]
base_block_type = lego_kb.ask(expr(f'TypeOf({baseblock},z)'))[z]
print(f'The base block is {baseblock} type of the base of {target} is {base_block_type}')
print('-'*20)
#########################################################
print('What are the bricks that are right of C?')
target='C'  # modify here
r=0
Bricks=[]
for b in BLOCKS:
    # print(type(lego_kb.ask(expr(f'Object_at_right({target},{b})'))), b, target)
    if (True if type(lego_kb.ask(expr(f'Object_at_right({target},{b})')))!=bool else False):
        r+=1
        if str(lego_kb.ask(expr(f'TypeOf({b},z)'))[z]) =='Brick':
            Bricks.append(b)
print(f'computer: there are total {r} blocks are right of {target}, {Bricks} are Brick')

print('-'*20)
#########################################################
print('What are all the bricks that are on top of i ?')
target='I'  # modify here
r=0
blocks=[]
for b in BLOCKS:
    search=lego_kb.ask(expr(f'Base({b},x)'))
    path = str(search.values())[13:len(str(search.values()))-2].replace(',', '').split(' ')
    if target in path:
        blocks.append(path[0])
        r+=1
r-=1
blocks=blocks[:-1]
print(f'computer: there are {r}, they are',blocks)

------------------------------q9---------------------------------
Is piece B on top of piece C?
computer:True
--------------------
What is the type and color of the piece on top of C?
It's a Tile and it's White
--------------------
What is the type of the base of H? 
The base block is J type of the base of H is Plate
--------------------
What are the bricks that are right of C?
computer: there are total 6 blocks are right of C, ['F', 'H', 'I'] are Brick
--------------------
What are all the bricks that are on top of i ?
computer: there are 2, they are ['G', 'H']


# QUESTION 10 (ADVANCED) 2 points


This question is more advanced and open ended. I provide two options. You only need to implement one 
of the two options to get full credit for this question. You are welcome to implement both but 
you will still get 1 point. 

## Option 1 

Extend the Legoworld knowledge base with a predicate to determine if a brick is part of an unstable 
tower. Any brick placed on top of a tile results in an unstable tower. For example the brown plate 
and grey brick in the middle are unstable but the red brick under the tile in the middle is stable. 
This is a trickier predicate to define so show a few cases to ensure that it works as expected. 

<img src="lego1.png" width="40%"/>

Provide a simple natural language processing interface to the LegoWorld that takes input from 
the user and returns the results in more natural lanuage. For example you could have the following 
dialog: 

* User: What color is brick B ? 
* Computer: Brick B is Red. 
* User: Is there a brick that is on top of a tile? 
* Computer: Yes, brick D is on top of a tile. 
etc 

You will basically write a simple translation layer from simplified English to FOL tell and ask requests and back to simplified English. 


This question is inspired by the classic natural understanding work using logic: https://en.wikipedia.org/wiki/SHRDLU



## Option 2 


This question explores the automatic constructions of a first-order logic knowledge base from a web resource and is more open ended than the other ones. The website https://www.songfacts.com/ contains a large variety of facts about music. Check the https://www.songfacts.com/categories link for some categories. Using selenium Python bindings https://selenium-python.readthedocs.io/ access the webpage and scrape at least three categories. Your code should scrape the information from the pages and convert it into relationships and facts in first-order logic using the syntax of expressions in logic.ipynb. Once you build your knowledge-base then write 4 non-trivial queries that show-case the expressiveness of FOL. These queries should not be possible to be answered easily using the web interface i.e they should have some logical connectives, more than one possible answer etc. The translation of the song facts from the web page to FOL should NOT be done by hand but using the web scraping tool you develop. You can use multiple cells in your answer.

In [72]:
print('------------------------------q9---------------------------------')

clauses=[]
BLOCKS =['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']

clauses.append(expr('TypeOf(A, Brick)'))
clauses.append(expr('TypeOf(B, Plate)'))
clauses.append(expr('TypeOf(C, Brick)'))
clauses.append(expr('TypeOf(D, Tile)'))
clauses.append(expr('TypeOf(E, Brick)'))
clauses.append(expr('TypeOf(F, Tile)'))
clauses.append(expr('TypeOf(G, Brick)'))
clauses.append(expr('TypeOf(H, Brick)'))
clauses.append(expr('TypeOf(I, Plate)'))

# COLOR
clauses.append(expr('Color(A, Red)'))
clauses.append(expr('Color(B, Brown)'))
clauses.append(expr('Color(C, Grey)'))
clauses.append(expr('Color(D, Brown)'))
clauses.append(expr('Color(E, Red)'))
clauses.append(expr('Color(F, Brown)'))
clauses.append(expr('Color(G, Red)'))
clauses.append(expr('Color(H, Blue)'))
clauses.append(expr('Color(I, Yellow)'))

# COLOR
clauses.append(expr('Stable(A, T)'))
clauses.append(expr('Stable(B, F)'))
clauses.append(expr('Stable(C, F)'))
clauses.append(expr('Stable(D, T)'))
clauses.append(expr('Stable(E, T)'))
clauses.append(expr('Stable(F, T)'))
clauses.append(expr('Stable(G, T)'))
clauses.append(expr('Stable(H, T)'))
clauses.append(expr('Stable(I, T)'))


# ONPLATE
clauses.append(expr('OnPlate(A)'))
clauses.append(expr('OnPlate(E)'))
clauses.append(expr('OnPlate(I)'))

# ON WITH CONNRCTION
clauses.append(expr('On(D, E)'))
clauses.append(expr('On(D, C)'))
clauses.append(expr('On(B, C)'))
clauses.append(expr('On(H, I)'))
clauses.append(expr('On(G, H)'))
clauses.append(expr('On(F, G)'))

# ON the left
clauses.append(expr('AtLeft(A, E)'))
clauses.append(expr('AtLeft(E, I)'))

# Base
clauses.append(expr('OnPlate(x) ==> Base(x,x)'))
clauses.append(expr('On(x,z) & Base (z,y) ==> Base(x,y)'))

# Base_at_right
clauses.append(expr('AtLeft(a, b) ==> Base_at_right(a, b)'))
clauses.append(expr('AtLeft(b, c) & Base_at_right(a, b) ==> Base_at_right(a, c)'))

# Object_at_right
clauses.append(expr('Base(x,a) & Base(y,b) & Base_at_right(a, b) ==> Object_at_right(x, y)'))

lego_kb1=FolKB(clauses)
# enter 'e' to end

def process_cmd(cmdin):
    cmd = cmdin.replace('?','').split(' ')
    
    
    if 'what' == cmd[0]:
        if cmd[1]=='color':
            block_color=lego_kb1.ask(expr(f'Color({cmd[-1].upper()},y)'))[y]
            print(f"it's {block_color}")
            return True
        if cmd[1]=='tpye':
            block_type=lego_kb1.ask(expr(f'TypeOf({cmd[-1].upper()},y)'))[y]
            print(f"it's {block_type}")
        return True
    if 'is'== cmd[0]:
        return True
    
    return False

# question may ask

# What color is brick
# What tpye of is brick


cmd = input('Enter qestion:')
while True:
    if cmd =='e':
        break
    if not process_cmd(cmd.lower()):
        print('I am not sure')
    cmd = input('Enter question:')
    

------------------------------q9---------------------------------
Enter qestion:e
